# Class 6: APIs

## APIs, conceptually

![Diagram showing how online payments work: Expedia talks to Delta, Delta talks to Stripe, Stripe talks to Visa, and Visa talks to Chase](extras/img/apis_conceptually/payments.png)

![Diagram showing how notifications flow through systems](extras/img/apis_conceptually/notifications.png)

![Diagram showing relationship between human languages, programming languages, and APIs](extras/img/apis_conceptually/languages.png)

interactions between systems ↔️

## Ways to get data

Method | How it happens | Pros | Cons
--- | :--- | :--- | :---
**Bulk** | Download, someone hands you a flash drive, etc. | Fast, one-time transfer | Can be large
**Scraping** | Data only available through a web site, PDF, or doc | You can turn anything into data | Tedious; fragile
**APIs** | If organization makes one available | Usually allows some filtering; can always pull latest-and-greatest | Requires network connection for every call; higher barrier to entry (reading documentation); subject to availability and performance of API

## Scraping

Common tools:

- [Beautiful Soup package](https://realpython.com/beautiful-soup-web-scraper-python/)
- [pandas' `read_html()`](https://pandas.pydata.org/docs/user_guide/io.html#html)

_Please pray to the Demo Gods that these all work and there's no profanity_

Pull table from [Wikipedia's list of countries by area](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area#Countries_and_dependencies_by_area):

In [1]:
import pandas as pd

tables = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area",
    match="Rank",
)
countries = tables[0]
countries

,Rank,Country / Dependency,Total in km2 (mi2),Land in km2 (mi2),Water in km2 (mi2),% water,Notes
0,NaN,World,"510,072,000 (196,940,000)","148,940,000 (57,510,000)","361,132,000 (139,434,000)",70.8,NaN
1,1,Russia,"17,098,246 (6,601,670)","16,378,410 (6,323,740)","719,836 (277,930)",4.21,"The largest country in the world, which spans ..."
2,NaN,Antarctica,"14,200,000 (5,500,000)","14,200,000 (5,500,000)",0 (0),0,"13,916,000 km2 (5,373,000 sq mi) (98%) of the ..."
3,2,Canada[Note 1],"9,984,670 (3,855,100)","9,093,507 (3,511,023)","891,163 (344,080)",8.93,Largest English- and largest French-speaking c...
4,3/4,China,"9,596,961 (3,705,407)","9,326,410 (3,600,950)","270,550 (104,460)",2.82,"Largest country entirely in Asia, and second-l..."
...,...,...,...,...,...,...,...
260,NaN,Ashmore and Cartier Islands (Australia),5 (1.9),5 (1.9),0 (0),0,NaN
261,NaN,Spratly Islands (disputed),(< 1.9),(< 1.9),0 (0),0,NaN
262,NaN,Coral Sea Islands (Australia),(< 1.2),(< 1.2),0 (0),0,NaN
263,194,Monaco,2.02 (0.78),2.02 (0.78),0 (0),0,European microstate. Smallest country with a c...


### Data is only available if it's available

## API calls in the wild

1. Go to [Candidates page on fec.gov](https://www.fec.gov/data/candidates/?has_raised_funds=true&is_active_candidate=true).
1. Right click and `Inspect`.
   - [More info about opening Developer Tools in various browsers.](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools#how_to_open_the_devtools_in_your_browser)
1. Go to the `Network` tab and reload.
1. Filter to `XHR`.
1. Click the API call.

We only see this because the tables on [fec.gov](https://fec.gov) are [rendered client-side](https://www.solutelabs.com/blog/client-side-vs-server-side-rendering-what-to-choose-when) using their JSON API. That won't be the case for all tables on all sites.

### Parts of a URL

![URL structure](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL/mdn-url-all.png)

[source](https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL#basics_anatomy_of_a_url)

For APIs:

- Often split into "base URL" + "endpoint"
- Endpoints are like function names: they represent the information you are retrieving or thing you are trying to do
- Parameters are like function arguments:
   - They allow options to be specified
   - Some are required, some are optional
   - They will differ from one endpoint/function to another
- Anchors won't be used

### API documentation

[FEC API](https://api.open.fec.gov/developers/)

### Try it out

1. Visit https://www.fec.gov/data/candidates/
1. In the Network tab's request list, right-click the API call.
1. Click `Open in New Tab`.
1. Replace the API key with `DEMO_KEY`.

## API calls from Python

Usually one of two ways:

- A software development kit (SDK) like [sodapy](https://pypi.org/project/sodapy/)
   - Abstracts the details away
   - Not available for all APIs
   - May have limitations
- [The `requests` package](https://docs.python-requests.org/) (nothing to do with 311 requests)

In [2]:
import requests

params = {"api_key": "DEMO_KEY", "q": "Jimmy McMillan", "sort": "-first_file_date"}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()
data

{'api_version': '1.0',
 'pagination': {'pages': 1, 'count': 2, 'page': 1, 'per_page': 20},
 'results': [{'district': '00',
   'first_file_date': '2015-10-13',
   'name': 'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH',
   'incumbent_challenge_full': 'Open seat',
   'inactive_election_years': None,
   'candidate_status': 'N',
   'incumbent_challenge': 'O',
   'election_years': [2016],
   'last_file_date': '2015-10-13',
   'election_districts': ['00'],
   'has_raised_funds': False,
   'office': 'P',
   'cycles': [2016, 2018],
   'party': 'REP',
   'district_number': 0,
   'active_through': 2016,
   'party_full': 'REPUBLICAN PARTY',
   'candidate_id': 'P60016805',
   'load_date': '2018-02-17T09:16:20+00:00',
   'federal_funds_flag': False,
   'state': 'US',
   'last_f2_date': '2015-10-13',
   'office_full': 'President',
   'flags': 'P60016805',
   'candidate_inactive': False},
  {'district': '00',
   'first_file_date': '1995-03-08',
   'name': 'MCMILLAN, JIMMY (AKA) JAMES ',
   'incumbent_challe

### Retrieving nested data

In [3]:
data["results"][0]["name"]

'MCMILLAN, JIMMY "RENT IS TOO DAMN HIGH'

### [In-class exercise](https://python-public-policy.afeld.me/en/{{school_slug}}/hw_5.ipynb)

### Reading into a DataFrame

In [4]:
params = {"api_key": "DEMO_KEY", "has_raised_funds": "true"}
response = requests.get("https://api.open.fec.gov/v1/candidates/", params=params)
data = response.json()

pd.DataFrame(data["results"])

,district,first_file_date,name,incumbent_challenge_full,inactive_election_years,candidate_status,incumbent_challenge,election_years,last_file_date,election_districts,...,active_through,party_full,candidate_id,load_date,federal_funds_flag,state,last_f2_date,office_full,flags,candidate_inactive
0,07,2021-12-27,"AADLAND, ERIK",Open seat,None,C,O,[2022],2022-08-10,[07],...,2022,REPUBLICAN PARTY,H2CO07170,2022-08-10T21:11:12+00:00,False,CO,2022-08-10,House,H2CO07170,False
1,03,2020-03-24,"AALDERS, TIM",Challenger,None,C,C,[2022],2022-03-21,[03],...,2022,REPUBLICAN PARTY,H2UT03280,2022-04-13T21:10:09+00:00,False,UT,2022-03-21,House,H2UT03280,False
2,00,2012-02-08,"AALDERS, TIMOTHY NOEL",Open seat,None,P,O,"[2012, 2018]",2018-04-23,"[00, 00]",...,2018,CONSTITUTION PARTY,S2UT00229,2019-03-27T16:02:41+00:00,False,UT,2018-04-23,Senate,S2UT00229,False
3,22,2019-10-17,"AALOORI, BANGAR REDDY",Open seat,None,C,O,[2020],2019-10-17,[22],...,2020,REPUBLICAN PARTY,H0TX22260,2020-03-18T21:13:37+00:00,False,TX,2019-10-17,House,H0TX22260,False
4,16,1976-04-12,"AAMODT, NORMAN O.",None,None,P,None,"[1976, 1978]",1978-07-05,"[16, 16]",...,1978,REPUBLICAN PARTY,H6PA16106,2002-03-30T00:00:00+00:00,False,PA,1978-07-05,House,H6PA16106,False
5,01,2012-02-22,"AANESTAD, SAMUEL",Challenger,None,P,C,[2012],2012-02-22,[01],...,2012,REPUBLICAN PARTY,H2CA01110,2013-04-26T09:04:30+00:00,False,CA,2012-02-22,House,H2CA01110,False
6,06,2017-04-26,"AARESTAD, DAVID",Challenger,None,C,C,[2018],2017-04-26,[06],...,2018,DEMOCRATIC PARTY,H8CO06237,2017-08-01T20:57:28+00:00,False,CO,2017-04-26,House,H8CO06237,False
7,00,2005-10-12,"AARON, LAURA DAVIS",Open seat,None,N,O,[2008],2007-03-13,[00],...,2008,DEMOCRATIC PARTY,P80002926,2016-11-17T06:10:48+00:00,False,US,2007-03-13,President,P80002926,False
8,32,2021-01-16,"AAZAMI, SHERVIN",Challenger,None,N,C,"[2022, 2024]",2022-07-15,"[32, 32]",...,2024,DEMOCRATIC PARTY,H2CA30291,2023-01-12T22:24:01+00:00,False,CA,2022-07-15,House,H2CA30291,False
9,07,2022-06-06,"ABAHSAIN, JILL",Challenger,None,C,C,[2022],2022-06-06,[07],...,2022,DEMOCRATIC-FARMER-LABOR,H2MN07162,2022-07-25T23:03:23+00:00,False,MN,2022-06-06,House,H2MN07162,False


## Back to 311 data

From [NYC Open Data Portal dataset page](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data), click `Export` -> `SODA API` -> `API Docs`.

### Most open data sites have APIs

Often built on platforms that provide them, e.g.

- [NYC Open Data Portal](https://opendata.cityofnewyork.us/) built on [Socrata](https://dev.socrata.com/)
- [data.gov built on CKAN](https://www.data.gov/developers/apis)

### Example: 311 requests from the last week

In [5]:
from datetime import datetime, timedelta

now = datetime.utcnow()
now

datetime.datetime(2023, 1, 26, 18, 24, 39, 570843)

In [6]:
start = now - timedelta(weeks=1)
start

datetime.datetime(2023, 1, 19, 18, 24, 39, 570843)

In [7]:
start.isoformat()

'2023-01-19T18:24:39.570843'

Using the [Socrata query language (SoQL)](https://dev.socrata.com/docs/queries/):

In [8]:
data_id = "erm2-nwe9"
params = {"$where": f"created_date between '{start.isoformat()}' and '{now.isoformat()}'"}

url = f"https://data.cityofnewyork.us/resource/{data_id}.json"
response = requests.get(url, params=params)
data = response.json()

data

[{'unique_key': '56612301',
  'created_date': '2023-01-25T12:00:00.000',
  'agency': 'DSNY',
  'agency_name': 'Department of Sanitation',
  'complaint_type': 'Derelict Vehicles',
  'descriptor': 'Derelict Vehicles',
  'location_type': 'Street',
  'incident_zip': '11356',
  'incident_address': '14-22 114 STREET',
  'street_name': '114 STREET',
  'cross_street_1': '14 AVENUE',
  'cross_street_2': '14 ROAD',
  'address_type': 'ADDRESS',
  'city': 'COLLEGE POINT',
  'status': 'Open',
  'resolution_description': 'The Department of Sanitation is in the process of investigating this complaint.',
  'resolution_action_updated_date': '2023-01-25T12:00:00.000',
  'community_board': '07 QUEENS',
  'bbl': '4040480013',
  'borough': 'QUEENS',
  'x_coordinate_state_plane': '1024891',
  'y_coordinate_state_plane': '225307',
  'open_data_channel_type': 'PHONE',
  'park_facility_name': 'Unspecified',
  'park_borough': 'QUEENS',
  'latitude': '40.78499841706239',
  'longitude': '-73.8532430054169',
  'lo

Like the FEC, Socrata uses their own API to populate the tables when browsing data on sites powered by them.

**At-home exercise:** Try filtering a table on the [NYC Open Data Portal](https://data.cityofnewyork.us/), and find the API calls that makes.

In [9]:
pd.DataFrame(data)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,...,landmark,closed_date,taxi_company_borough,taxi_pick_up_location,facility_type,bridge_highway_name,bridge_highway_segment,bridge_highway_direction,road_ramp,due_date
0,56612301,2023-01-25T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11356,14-22 114 STREET,114 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56613323,2023-01-25T02:00:06.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11378,60-32 59 AVENUE,59 AVENUE,...,59 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56610349,2023-01-25T01:58:15.000,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11372,37-30 93 STREET,93 STREET,...,93 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56615296,2023-01-25T01:58:07.000,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11377,32-28 68 STREET,68 STREET,...,68 STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,56611871,2023-01-25T01:57:42.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11212,61 THATFORD AVENUE,THATFORD AVENUE,...,THATFORD AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,56610014,2023-01-24T21:25:52.000,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10459,1302 REV JAMES POLITE AVENUE,REV JAMES POLITE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,56613866,2023-01-24T21:25:48.000,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10452,1434 OGDEN AVENUE,OGDEN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,56610198,2023-01-24T21:25:47.000,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,Street/Sidewalk,11106,33 STREET,33 STREET,...,NaN,2023-01-24T21:48:09.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,56609627,2023-01-24T21:25:39.000,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10034,98 COOPER STREET,COOPER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Coincidence there were exactly 1,000 results?

### Pagination

- Most APIs limit the number of results returned.
- [Socrata defaults to 1,000.](https://dev.socrata.com/docs/queries/limit.html)
- Need to use a loop with parameters like [`$limit`](https://dev.socrata.com/docs/queries/limit.html)+[`$offset`](https://dev.socrata.com/docs/queries/offset.html) (Socrata) or `page`+`per_page` ([FEC](https://api.open.fec.gov/developers/))
   - [`append()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html) to DataFrame

## Things are going to differ by API

- Endpoints
- Supported parameters
- Response structure
   - [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) can help
- Quality of documentation
- Helpfulness of errors
- Size/helpfulness of community

Gotta read and experiment.

## Homework 6

In real/ideal world, start with specific question and find data to answer it:

![project flow](extras/img/projectflow.png)

_Source: [Big Data and Social Science](https://textbook.coleridgeinitiative.org/chap-intro.html#the-structure-of-the-book)_

Data needed often doesn't exist or is hard (or impossible) to find/access

![project flow](extras/img/projectflow_amended.png)

[Homework 6](https://python-public-policy.afeld.me/en/{{school_slug}}/hw_6.ipynb)

## No homework/resubmissions will be accepted after Wednesday 12/21 at 6:45pm ET

In other words, Homework 6 cannot be late.